# Psycopg

Let's import `psycopg` and make a simple query. This requires:
- preparing connection string (see [Postgres docs](https://www.postgresql.org/docs/current/libpq-connect.html#LIBPQ-CONNSTRING))
- making a connection
- creating a cursor
- running a query on the cursor
- parsing results

See the code below and comments. As our query, we will read:
- total number of trains stopping in Amsterdam
- all trains that were late in Amsterdam

In [ ]:
import psycopg

# build the connection string
host = "localhost"
username = "postgres"
password = "postgres"  # NEVER do this in real code, read from secure secrets file
db_name = "postgres"

db_conn_string = f"host={host} user={username} password={password} dbname={db_name}"
print(db_conn_string)

In [ ]:
query_total_trains = """
SELECT COUNT(*)
FROM services
WHERE "Stop:Station code" = 'ASD'
"""

query_late_departure_trains = """
SELECT *
FROM services
WHERE "Stop:Station code" = 'ASD' AND "Stop:Departure delay" > 0
"""

# connect to the database (context manager will automatically close connection)
with psycopg.connect(db_conn_string) as conn:
    # create a cursor to execute commands
    with conn.cursor() as cur:
        # send query and fetch result
        cur.execute(query_total_trains)
        total_trains = cur.fetchone()

        # send query and fetch results
        cur.execute(query_late_departure_trains)
        late_trains = cur.fetchall()

print("Total trains:", total_trains)

print("Late trains:")
for record in late_trains[:3]:
    print(record)

**Connection** manages database session and network connection. It should always be used as a context manager, to close it after usage, since keeping "dangling" open connections wastes system resources. Opening and closing connections has some cost, so for batches of tasks it's good to keep the connection open.

**Connection string** defines the Postgres connection. We can either build a string, or pass keyword arguments to `.connect()`.

**Cursor** executes commands to the database. It will always use a transaction in psycopg. If you don't explicitly commit by calling `conn.commit()` inside the cursor block, it will roll back all changes. However, here we only read data, so this doesn't matter.

**Results** are by default returned as a tuple for `.fetchone()` (even a single element), or list of tuples for `.fetchall()`. Types are automatically parsed, e.g. dates are `datetime.date` objects, with exact rules [in the documentation](https://www.psycopg.org/psycopg3/docs/basic/adapt.html).

Tuples are not always convenient, and we have two main options:
- use [row factory](https://www.psycopg.org/psycopg3/docs/advanced/rows.html#row-factories) to return dictionaries, named tuples, or a custom dataclass (you need to write it first)
- use an ORM like SQLAlchemy or peewee

Which one makes the most sense depends on a use case. If you use a custom dataclass, you are really close to using an ORM, so the line is often blurry.

**Efficiency** typically has tradeoffs for large data. Processing data in sizes is typically necessary, as we can't easily load all data into memory like here. Smaller batches are sometimes more efficient within the database, particularly when we have proper indexes. For example, you may want to query for a single month at a time. However, too small batches will waste resources on network requests.

**Column names** are often useful for automation, and we can get them without wasting a lot of memory for dictionaries. In fact, creating Pandas DataFrames from default tuples is actually faster and easier. We can get the list of column names from the cursor `description` attribute:

In [ ]:
with psycopg.connect(db_conn_string) as conn:
    with conn.cursor() as cur:
        # LIMIT 0 for efficiency - we don't want rows, just column names
        cur.execute("SELECT * FROM services LIMIT 0")
        colnames = [desc[0] for desc in cur.description]
        print(colnames)

**Exercise**


1. Write a query to get all trains that both arrived and departured late.

2. Use a row factory to return dictionaries. 
```python
    (from psycopg.rows import dict_row)
```

3. Save the result to file `data/late_train.jsonl` in JSON Lines format. `json` module will be useful.

Since JSON does not have a default date or datetime format, we need to provide an explicit encoder for that. Pass this class to `json.dumps()` as `cls` argument.

In [ ]:
import datetime
import json


class DateEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.date, datetime.datetime)):
            return obj.isoformat()
        return json.JSONEncoder.default(self, obj)

In [ ]:
# Your solution ...